In [5]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import warnings
warnings.simplefilter('ignore', category=UserWarning)

from Processing.load_dataset import Load_dataset
from Processing.processing import Split, Valid, Submission
from Engineering.enginnering import engin
from Valid.validation import rmse

from Model.lightgbm import LightGBM
from Model.xgboost import XGBOOST
from sklearn.metrics import mean_squared_error

In [3]:
train, test = Load_dataset()

# 加工・処理
train_df = engin(train)

# val, train, test
X_train, X_test, y_train, y_test, index = Split(train_df)
X_train, X_valid, y_train, y_valid = Valid(X_train, y_train)

## light-gbm

In [ ]:
# 学習
Light_GBM = LightGBM(X_train, X_valid, y_train, y_valid, fig=1)
importance = pd.DataFrame(sorted(Light_GBM.feature_importance(), reverse=True), index=X_test.columns, columns=['importance'])
plt.figure(figsize=(10,6))
plt.bar(X_test.columns[:10], np.ravel(importance.values)[:10])

In [ ]:
# 評価
test_predict = Light_GBM.predict(X_test)
rmse(test_predict, y_test)

In [ ]:
# create
test_df = engin(test)
index = np.array(test_df["id"])
test_df = test_df.drop(columns=["id"])
predict_light_gbm = Light_GBM.predict(data=test_df)

# 提出
Submission(index, predict_light_gbm, name='4')

## xgboost

In [4]:
xgboost = XGBOOST(X_train, X_valid, y_train, y_valid, fig=1)

[23:16:05] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:573: 
Parameters: { "random_seed" } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.




In [25]:
importance = list(xgboost.get_score(importance_type='weight').values())
col = list(xgboost.get_score(importance_type='weight').keys())
plt.figure(figsize=(10,6))
plt.bar(col, importance)

In [35]:
# 評価
import xgboost as xgb
dm_test = xgb.DMatrix(X_test)
test_predict = xgboost.predict(dm_test)
rmse(test_predict, y_test)

test_df = engin(test)
index = np.array(test_df["id"])
test_df = test_df.drop(columns=["id"])
test = xgb.DMatrix(test_df)

predict_xgboost = xgboost.predict(test, ntree_limit=xgboost.best_ntree_limit)
Submission(index, predict_xgboost, name='xgb_1')

20.395146661257535
